In [128]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [129]:
with open('data.yaml',mode='r') as f:
    data_yaml=yaml.load(f,Loader=SafeLoader)

labels=data_yaml['names']
print(labels)

['crater', 'boulder']


In [130]:
yolo=cv2.dnn.readNetFromONNX('C:/Users/DELL/OneDrive/Desktop/yolo/prediction/Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [177]:
img=cv2.imread('C:/Users/DELL/OneDrive/Desktop/yolo/prediction/moon_img_1.jpg')
image=img.copy()
# cv2.imshow('image',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
row,col,d=image.shape

In [178]:
max_rc=max(row,col)
input_image=np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col]=image

INPUT_WH_YOLO=640
blob=cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds=yolo.forward()

# cv2.imshow('input_image',input_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [179]:
print(preds.shape)

(1, 25200, 7)


In [180]:
detections=preds[0]
boxes=[]
confidences=[]
classes=[]

image_w,image_h=input_image.shape[:2]
x_factor=image_w/INPUT_WH_YOLO
y_factor=image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row=detections[i]
    confidence=row[4]
    if confidence>0.3:
        class_score=row[5:].max()
        class_id=row[5:].argmax()

        if class_score>0.15:
            cx,cy,w,h=row[0:4]
            left=int((cx-0.5*w)*x_factor)
            top=int((cy-0.5*h)*y_factor)
            width=int(w*x_factor)
            height= int(h*y_factor)
            box=np.array([left,top,width,height])
    
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

boxes_np=np.array(boxes).tolist()
confidences_np=np.array(confidences).tolist()

index=cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()

In [181]:
for ind in index:
    x,y,w,h=boxes_np[ind]
    bb_conf=int(confidences_np[ind]*100)
    classes_id=classes[ind]
    class_name=labels[classes_id]

    text=f'{class_name}:{bb_conf}%'
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)

    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

In [182]:
cv2.imshow('original',img)
cv2.imshow('yolo_prediction',image)
cv2.waitKey(0)
cv2.destroyAllWindows()